# Introduction

We tackle the problem of OCR post processing. In OCR, we map the image form of the document into the text domain. This is done first using an CNN+LSTM+CTC model, in our case based on tesseract. Since this output maps only image to text, we need something on top to validate and correct language semantics.

The idea is to build a language model, that takes the OCRed text and corrects it based on language knowledge. The langauge model could be:
- Char level: the aim is to capture the word morphology. In which case it's like a spelling correction system.
- Word level: the aim is to capture the sentence semnatics. But such systems suffer from the OOV problem.
- Fusion: to capture semantics and morphology language rules. The output has to be at char level, to avoid the OOV. However, the input can be char, word or both.

The fusion model target is to learn:

    p(char | char_context, word_context)

In this workbook we use seq2seq vanilla Keras implementation, adapted from the lstm_seq2seq example on Eng-Fra translation task. The adaptation involves:

- Adapt to spelling correction, on char level
- Pre-train on a noisy, medical sentences
- Fine tune a residual, to correct the mistakes of tesseract 
- Limit the input and output sequence lengths
- Enusre teacher forcing auto regressive model in the decoder
- Limit the padding per batch
- Learning rate schedule
- Bi-directional LSTM Encoder
- Bi-directional GRU Encoder


# Imports

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from keras.models import load_model
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from autocorrect import spell
%matplotlib inline

Using TensorFlow backend.


# Utility functions

In [2]:
# Limit gpu allocation. allow_growth, or gpu_fraction
def gpu_alloc():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc()

In [4]:
def calculate_WER_sent(gt, pred):
    '''
    calculate_WER('calculating wer between two sentences', 'calculate wer between two sentences')
    '''
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')
    d = np.zeros(((len(gt_words) + 1), (len(pred_words) + 1)), dtype=np.uint8)
    # d = d.reshape((len(gt_words)+1, len(pred_words)+1))

    # Initializing error matrix
    for i in range(len(gt_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(gt_words) + 1):
        for j in range(1, len(pred_words) + 1):
            if gt_words[i - 1] == pred_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(gt_words)][len(pred_words)]

In [5]:
def calculate_WER(gt, pred):
    '''

    :param gt: list of sentences of the ground truth
    :param pred: list of sentences of the predictions
    both lists must have the same length
    :return: accumulated WER
    '''
#    assert len(gt) == len(pred)
    WER = 0
    nb_w = 0
    for i in range(len(gt)):
        #print(gt[i])
        #print(pred[i])
        WER += calculate_WER_sent(gt[i], pred[i])
        nb_w += len(gt[i])

    return WER / nb_w

In [6]:
def load_data_with_gt(file_name, num_samples, max_sent_len, min_sent_len, delimiter='\t', gt_index=1, prediction_index=0):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []
    gt_texts = []
    target_texts = []
    for row in open(file_name, encoding='utf8'):
        if cnt < num_samples :
            #print(row)
            sents = row.split(delimiter)
            input_text = sents[prediction_index]
            
            target_text = '\t' + sents[gt_index] + '\n'
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len and len(target_text) > min_sent_len and len(target_text) < max_sent_len:
                cnt += 1
                
                input_texts.append(input_text)
                target_texts.append(target_text)
                gt_texts.append(sents[gt_index])
    return input_texts, target_texts, gt_texts

In [7]:
def load_data(file_name, num_samples, max_sent_len, min_sent_len):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []   
    
    #for row in open(file_name, encoding='utf8'):
    for row in open(file_name):
        if cnt < num_samples :            
            input_text = row           
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len:
                cnt += 1                
                input_texts.append(input_text)
    return input_texts

In [8]:
def vectorize_data(input_texts, max_encoder_seq_length, num_encoder_tokens, vocab_to_int):
    
    if(len(input_texts) > max_encoder_seq_length):
        input_texts = input_texts[:max_encoder_seq_length]
    
    '''Prepares the input text and targets into the proper seq2seq numpy arrays'''
    encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
    
    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text[:max_encoder_seq_length]):
            # c0..cn
            encoder_input_data[i, t] = vocab_to_int[char]
                
    return encoder_input_data

In [9]:
def decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, max_decoder_seq_length, vocab_to_int, int_to_vocab):
    
    #print(max_decoder_seq_length)
    # Encode the input as state vectors.
    encoder_outputs, h, c  = encoder_model.predict(input_seq)
    states_value = [h,c]
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    #print(input_seq)
    attention_density = []
    i = 0
    special_chars = ['\\', '/', '-', '—' , ':', '[', ']', ',', '.', '"', ';', '%', '~', '(', ')', '{', '}', '$']
    #special_chars = []
    while not stop_condition:
        #print(target_seq)
        output_tokens, attention, h, c  = decoder_model.predict(
            [target_seq, encoder_outputs] + states_value)
        #print(attention.shape)
        attention_density.append(attention[0][0])# attention is max_sent_len x 1 since we have num_time_steps = 1 for the output
        # Sample a token
        #print(output_tokens.shape)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        #print(sampled_token_index)
        sampled_char = int_to_vocab[sampled_token_index]
        
        orig_char = int_to_vocab[int(input_seq[:,i][0])]
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            #print('End', sampled_char, 'Len ', len(decoded_sentence), 'Max len ', max_decoder_seq_length)
            sampled_char = ''
        
        # Copy digits as it, since the spelling corrector is not good at digit corrections
        if(orig_char.isdigit() or orig_char in special_chars):
            decoded_sentence += orig_char            
        else:
            if(sampled_char.isdigit() or sampled_char in special_chars):
                decoded_sentence += ''
            else:
                decoded_sentence += sampled_char
        
        #decoded_sentence += sampled_char


        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
        
        i += 1
        if(i > 48):
            i = 0
    attention_density = np.array(attention_density)
    
    # Word level spell correct
    
    corrected_decoded_sentence = ''
    for w in decoded_sentence.split(' '):
        corrected_decoded_sentence += spell(w) + ' '
    decoded_sentence = corrected_decoded_sentence
    
    return decoded_sentence, attention_density


# Load data

# Load model params

In [10]:
data_path = '../../dat/'

In [11]:
max_sent_lengths = [50, 100]

In [12]:
vocab_file = {}
model_file = {}
encoder_model_file = {}
decoder_model_file = {}
model = {}
encoder_model = {}
decoder_model = {}
vocab = {}
vocab_to_int = {}
int_to_vocab = {}
max_sent_len = {}
min_sent_len = {}
num_decoder_tokens = {}
num_encoder_tokens = {}
max_encoder_seq_length = {}
max_decoder_seq_length = {}

In [13]:

for i in max_sent_lengths:
    vocab_file[i] = 'vocab-{}.npz'.format(i)
    model_file[i] = 'best_model-{}.hdf5'.format(i)
    encoder_model_file[i] = 'encoder_model-{}.hdf5'.format(i)
    decoder_model_file[i] = 'decoder_model-{}.hdf5'.format(i)
    
    vocab = np.load(file=vocab_file[i])
    vocab_to_int[i] = vocab['vocab_to_int'].item()
    int_to_vocab[i] = vocab['int_to_vocab'].item()
    max_sent_len[i] = vocab['max_sent_len']
    min_sent_len[i] = vocab['min_sent_len']
    input_characters = sorted(list(vocab_to_int))
    num_decoder_tokens[i] = num_encoder_tokens[i] = len(input_characters) #int(encoder_model.layers[0].input.shape[2])
    max_encoder_seq_length[i] = max_decoder_seq_length[i] = max_sent_len[i] - 1#max([len(txt) for txt in input_texts])
    
    model[i] = load_model(model_file[i])
    encoder_model[i] = load_model(encoder_model_file[i])
    decoder_model[i] = load_model(decoder_model_file[i])

/opt/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
num_samples = 1000000
#tess_correction_data = os.path.join(data_path, 'test_data.txt')
#input_texts = load_data(tess_correction_data, num_samples, max_sent_len, min_sent_len)

OCR_data = os.path.join(data_path, 'new_trained_data.txt')
#input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len, min_sent_len, delimiter='|',gt_index=0, prediction_index=1)
input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len=10000, min_sent_len=0)

In [15]:
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])

1951
Me dieal Provider Roles: Treating  
 	Medical Provider Roles: Treating


Provider First Name: Christine  
 	Provider First Name: Christine


Provider Last Name: Nolen, MD  
 	Provider Last Name: Nolen, MD


Address Line 1 : 7 25 American Avenue  
 	Address Line 1 : 725 American Avenue


City. W’aukesha  
 	City: Waukesha


StatefProvinee: ‘WI  
 	State/Province: WI


Postal Code: 5 31 88  
 	Postal Code: 53188


Country". US  
 	Country:  US


Business Telephone: (2 62) 92 8- 1000  
 	Business Telephone: (262) 928- 1000


Date ot‘Pirst Visit: 1 2/01f20 17  
 	Date of First Visit: 12/01/2017




In [16]:
# Spell correct before inference
'''
input_texts_ = []
for sent in input_texts:
    sent_ = ''
    for word in sent.split(' '):
        sent_ += spell(word) + ' '
    input_texts_.append(sent_)
input_texts = input_texts_
input_texts_ = []
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])
'''

"\ninput_texts_ = []\nfor sent in input_texts:\n    sent_ = ''\n    for word in sent.split(' '):\n        sent_ += spell(word) + ' '\n    input_texts_.append(sent_)\ninput_texts = input_texts_\ninput_texts_ = []\n# Sample data\nprint(len(input_texts))\nfor i in range(10):\n    print(input_texts[i], '\n', target_texts[i])\n"

In [17]:
decoded_sentences = []

#for seq_index in range(len(input_texts)):


for i, input_text in enumerate(input_texts):
    #print(input_text)
    # Find the input length range to choose the proper model to use
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])
    
    

    target_text = gt_texts[i]
    
    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)
    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    
    print('-')
    print('Input sentence:', input_text)
    print('GT sentence:', target_text)
    print('Decoded sentence:', decoded_sentence)   
    decoded_sentences.append(decoded_sentence)
    

    

-
Input sentence: Me dieal Provider Roles: Treating 
GT sentence: Medical Provider Roles: Treating

Decoded sentence: Medical Provider Roles:Treating a 
-
Input sentence: Provider First Name: Christine 
GT sentence: Provider First Name: Christine

Decoded sentence: Provider First Name Christine a 
-
Input sentence: Provider Last Name: Nolen, MD 
GT sentence: Provider Last Name: Nolen, MD

Decoded sentence: Provider Last Name Dolens MD a 
-
Input sentence: Address Line 1 : 7 25 American Avenue 
GT sentence: Address Line 1 : 725 American Avenue

Decoded sentence: Address Line a a 725 American Avenue 
-
Input sentence: City. W’aukesha 
GT sentence: City: Waukesha

Decoded sentence: City.Stakes Week 
-
Input sentence: StatefProvinee: ‘WI 
GT sentence: State/Province: WI

Decoded sentence: StateProvinee: WI I N S 
-
Input sentence: Postal Code: 5 31 88 
GT sentence: Postal Code: 53188

Decoded sentence: Postal Code a 3188 a 
-
Input sentence: Country". US 
GT sentence: Country:  US

Decoded

-
Input sentence: [:I Piease Check box if address is inoorreot or insurawce information has changed andmdicatechange(s)0n revalse side. 
GT sentence: Please check if address is incorrect or insurance information has changed and indicate charge(s) on reverse side

Decoded sentence: these Check box indurated in ore insurant and in[:race information has changed and dis cantechang[: 
-
Input sentence: - VIASTERCARD
GT sentence: MASTERCARD

Decoded sentence: MASTERCARD 
-
Input sentence: DISCOVER
GT sentence: DISCOVER

Decoded sentence: DISCOVER 
-
Input sentence: VISA
GT sentence: VISA

Decoded sentence: VISA 
-
Input sentence: RATIENTIIAME
GT sentence: PATIENT NAME

Decoded sentence: PATIENTS SAME 
-
Input sentence: DUEDATE 
GT sentence: DUE DATE

Decoded sentence: DUE DATE 
-
Input sentence: GUARANTORID
GT sentence: GUARANTOR ID

Decoded sentence: GUARANTOR ID 
-
Input sentence: BALANCE DUE
GT sentence: BALANCE DUE

Decoded sentence: BALANCE DUE 
-
Input sentence: Amount Enclosed 
GT sen

-
Input sentence: I authorize the followin persons: health care professionals, hospitals, clinics, laboratories, pharmacies and all other medical or me ically related providers, facilities or services, rehabilitation professionals, vocational evaluators, health plans, insurance companies, third party administrators, insurance producers, insurance service providers, consumer reporting agencies including credit bureaus, GENEX Services, Inc., The Advocator Group and other Social Security advocacy vendors, professional licensing bodies, employers, attorneys, financial institutions and/or banks, and governmental entities; 
GT sentence: I authorize the following persons: health care professionals, hospitals, clinics, laboratories, pharmacies and all other medical or medically related providers, facilities or services, rehabilitation professionals, vocational evaluators, health plans, insurance companies, third party administrators, insurance producers, insurance service providers, consumer r

-
Input sentence: I signed on behalf of the Insured as (Relationship). If Power of Attorney Designee, Guardian, or Conservator, please attach a copy of the document granting authority.
GT sentence: I signed on behalf of the Insured as (Relationship). If Power of Attorney Designee, Guardian, or Conservator, please attach a copy of the document granting authority.

Decoded sentence: Is inkled on bleed as Relationship of the Insured as melinae Designed Describe guardian Cor Contain a 
-
Input sentence: Unum is a registered trademark and marketing brand of Unum Group and its insuring subsidiaries. 
GT sentence: Unum is a registered trademark and marketing brand of Unum Group and its insuring subsidiaries.

Decoded sentence: Unum is a registered trademark and marketing brand of Unum Group and its insuring subsidiaries 
-
Input sentence: CL-1116 ( 
GT sentence: CL-1116

Decoded sentence: CL-1116 d 
-
Input sentence: Daytime Phone: 
GT sentence: Daytime Phone:

Decoded sentence: Daytime Phone

-
Input sentence: Date ofVisithdmission 
GT sentence: Date of Visit/Admission: 12/01/2017

Decoded sentence: Date of VisitAdmission a 
-
Input sentence: Date ot’DischaIge: 1210112017 
GT sentence: Date of Discharge: 12/01/2017

Decoded sentence: Date of Discharge 1210112017 
-
Input sentence: Proc edure : Cleaning, may: bandage
GT sentence: Procedure: Cleaning, xray, bandage

Decoded sentence: Procedure leaning and bandage 
-
Input sentence: E11113] arm en t In formation 
GT sentence: Employment Information

Decoded sentence: E11113] ardent Information 
-
Input sentence: Employer Name: 
GT sentence: Employer Name:

Decoded sentence: Employer Name a 
-
Input sentence: Electronic Submission 
GT sentence: Electronic Submission

Decoded sentence: Electronic Submission 
-
Input sentence: Claim Event Identiﬁer: 
GT sentence: Claim Event Identifier:

Decoded sentence: Claim Event Identified a 
-
Input sentence: Submission Date: 03411419018
GT sentence: Submission Date: 03/14/2018

Decoded sen

...

-
Input sentence: Medical Provider Specially. Podiﬂll'ibl 
GT sentence: Medical Provider Specially: Podiatrist

Decoded sentence: Medical Provider Specialty Podicible 
-
Input sentence: Medical Provider Roles: [rearing 
GT sentence: Medical Provider Roles: Treating

Decoded sentence: Medical Provider Roles rearing a 
-
Input sentence: Provider First Name: Ryan 
GT sentence: Provider First Name: Ryan

Decoded sentence: Provider First Name Ryan 
-
Input sentence: Pl'mider la st Name: Kish 
GT sentence: Provider Last Name: Kish

Decoded sentence: Provider Last Name:Kish a 
-
Input sentence: Address Line 1: 3905. W’est Sylvania Ave. 
GT sentence: Address Line 1: 3905 West Sylvania Ave.

Decoded sentence: Address Line of 3905.eldania Sellate a 
-
Input sentence: City: Toledo 
GT sentence: City: Toledo

Decoded sentence: City Toled a 
-
Input sentence: Stater'Proxince: OH 
GT sentence: State/Province: OH

Decoded sentence: StateProvince:HOH 
-
Input sentence: Claim Tji'pe: VB Accident - Acci

-
Input sentence: Electronically Signed Date: Monday, 03:"12/2018 08:04 AM 
GT sentence: Electronically Signed Date: Monday, 03/12/2018 08:04 AM

Decoded sentence: Electronically Signed Date Monday 03:"12/20180 a 
-
Input sentence: Spou s 0 Information 
GT sentence: Spouse Information

Decoded sentence: Spouse information 
-
Input sentence: First Name: 
GT sentence: First Name:

Decoded sentence: First Name a 
-
Input sentence: Middle Name/initial: 
GT sentence: Middle Name/Initial:

Decoded sentence: Middle Name/Initial: a 
-
Input sentence: Last Name: 
GT sentence: Last Name:

Decoded sentence: Last Name a 
-
Input sentence: Social Security Number: 
GT sentence: Social Security Number:

Decoded sentence: Social Security Number a 
-
Input sentence: Birth Date: 
GT sentence: Birth Date:

Decoded sentence: Birth Date a 
-
Input sentence: Gender: 
GT sentence: Gender:

Decoded sentence: Gender a 
-
Input sentence: Claim Event Information 
GT sentence: Claim Event Information

Decoded sen

-
Input sentence: Address Line 1 2142 N Cove Blvd  
GT sentence: Address Line 1 : 2142 N Cove Blvd

Decoded sentence: Address Line a 2142 Cover Give a 
-
Input sentence: C ity. OH 
GT sentence: City: OH

Decoded sentence: City.OH NAME 
-
Input sentence: S lute riProx-inee: 
GT sentence: State/Province: OH

Decoded sentence: Saluter province a 
-
Input sentence: Postal Code: 43606 
GT sentence: Postal Code: 43606

Decoded sentence: Postal Code 43606 a 
-
Input sentence: (10th US 
GT sentence: Country: US

Decoded sentence: (10 US S 
-
Input sentence: Date ofVisit/A d‘miss ion 03/09/2018 
GT sentence: Date of Visit/Admission: 03/09/2018

Decoded sentence: Date of Visi/Admiss on 03/09/201 
-
Input sentence: Date ofDiseharge: 03/09/2018 
GT sentence: Date of Discharge: 03/09/2018

Decoded sentence: Date of Discharg:03/09/2018 
-
Input sentence: Procedure: ER visit, me 
GT sentence: Procedure: ER visit, Xray

Decoded sentence: Procedure ER visit,Number a 
-
Input sentence: Emplomi ent In fo

-
Input sentence: To Unum Group and its subsidiaries, Unum Life Insurance Company ofAmerica, Provident Life and Accident Insurance Compan , The Paul Revere Life Insurance Company, and persons who evaluate claims for any of those companies (“ num"); 
GT sentence: To Unum Group and its subsidiaries, Unum Life Insurance Company of America, Provident Life and Accident Insurance Company, The Paul Revere Life Insurance Company, and persons who evaluate claims for any of those companies (“Unum”);

Decoded sentence: To Unum Group and its subsidiaries Unum Life Insurance Complice Insurance Complete,and Accident Con 
-
Input sentence: So that Unum may evaluate and administer my claims, including providing assistance with return to work. For such evaluation and administration of claims, this authorization is valid for two years, or the duration of my claim for benefits, whichever is shorter. I understand that once My Information is disclosed to Unum, any privacy rotections established by HIPAA ma

-
Input sentence: 15 Surgery Required: No 
GT sentence: Is Surgery Required: No

Decoded sentence: of Surgery Required No a 
-
Input sentence: Medical Proxitler Information , Physician 
GT sentence: Medical Provider Information - Physician

Decoded sentence: Medical Provider Information Physician 
-
Input sentence: Medical Provider Specially. Orthopedic Surgeon 
GT sentence: Medical Provider Specially: Orthopedic Surgeon

Decoded sentence: Medical Provider Specialty Orthopedic Surgeon a 
-
Input sentence: unum‘ 
GT sentence: unum

Decoded sentence: unum 
-
Input sentence: June 19, 2012 
GT sentence: June 19, 2012

Decoded sentence: June 19, 2012 
-
Input sentence: Conﬁrmation of Coveraoe 
GT sentence: Confirmation of Coverage

Decoded sentence: Confirmation of Coverage 
-
Input sentence: The information below is provided to give you a general summary of your coverage and premium consistent with the benefits outlined in your certiﬁcate. 
GT sentence: The information below is provided to

In [18]:
WER_spell_correction = calculate_WER(gt_texts, decoded_sentences)
print('WER_spell_correction |TEST= ', WER_spell_correction)

WER_spell_correction |TEST=  0.130755891233


In [19]:
WER_OCR = calculate_WER(gt_texts, input_texts)
print('WER_OCR |TEST= ', WER_OCR)

WER_OCR |TEST=  0.0563287295006
